## Instalacion de librerias

In [ ]:
!pip install langchain langchain-community faiss-cpu pypdf tiktoken langchain-nvidia-ai-endpoints -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install tiktoken -q

## Implementacion de rag con FAISS

In [ ]:
# 1. Configuración inicial
import os
from dotenv import load_dotenv
from config.api_config import NVIDIA_API_KEY, PDF_PATH

# 2. Cargar dependencias
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
import tiktoken

# 3. Función para conteo de tokens
def count_tokens(text: str) -> int:
    tokenizer = tiktoken.get_encoding("cl100k_base")
    return len(tokenizer.encode(text))

# 4. Procesamiento de documentos
def process_documents(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=30,
        length_function=count_tokens,
        separators=["\n\n", "\n", ". ", "! ", "? ", "; ", " ", ""]
    )
    
    return text_splitter.split_documents(documents)

# 5. Creación de vectorstore
def create_vectorstore(splits):
    embedder = NVIDIAEmbeddings()
    return FAISS.from_documents(documents=splits, embedding=embedder)

# 6. Configuración de la cadena QA
def setup_qa_chain(vectorstore):
    llm = ChatNVIDIA(model="nvidia/llama-3.3-nemotron-super-49b-v1")
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        return_source_documents=True
    )

# 7. Interfaz de usuario
def run_chat(qa_chain):
    def format_source(doc):
        page = doc.metadata.get('page', 0) + 1
        content = doc.page_content[:100].replace('\n', ' ')
        return f"Página {page}: {content}..."

    print("🌟 Asistente RAG - Basado en tu PDF 📄\n")
    while True:
        try:
            query = input("\n🤔 Tu pregunta (o 'salir'): ").strip()
            if query.lower() in ['salir', 'exit', 'quit']:
                break

            result = qa_chain.invoke({"query": query})
            
            print("\n💡 Respuesta:")
            print(result["result"])

            if result.get("source_documents"):
                print("\n📚 Fuentes relevantes:")
                for i, doc in enumerate(result["source_documents"][:2], 1):
                    print(f"{i}. {format_source(doc)}")

        except Exception as e:
            print(f"\n⚠️ Error: {str(e)}")
            continue

# 8. Ejecución principal
if __name__ == "__main__":
    # Cargar configuración
    load_dotenv()
    
    # Pipeline completo
    splits = process_documents(PDF_PATH)
    vectorstore = create_vectorstore(splits)
    qa_chain = setup_qa_chain(vectorstore)
    
    # Iniciar chat
    run_chat(qa_chain)

/usr/local/lib/python3.11/dist-packages/langchain_nvidia_ai_endpoints/_common.py:217: UserWarning: Found nvidia/llama-3.3-nemotron-super-49b-v1 in available_models, but type is unknown and inference may fail.
  warnings.warn(


🌟 Asistente RAG - Basado en tu PDF 📄


🤔 Tu pregunta (o 'salir'): cual es el objetivo general 

💡 Respuesta:
Basándome en el contexto proporcionado, puedo responder directamente a tu pregunta:

**Objetivo General (1.3.1.)**

* **Descripción**: Desarrollar una herramienta basada en inteligencia artificial que sintetice información a partir de links de YouTube y documentos en formato PDF, DOCX y PPT, permitiendo la generación de resúmenes automáticos y cuestionarios interactivos para el apoyo del aprendizaje.

En resumen, el objetivo general es **desarrollar una herramienta de IA para sintetizar contenido de múltiples fuentes (YouTube, PDF, DOCX, PPT) y generar resúmenes y cuestionarios para apoyar el aprendizaje**.

📚 Fuentes relevantes:
1. Página 15: 1.3.1. Objetivo general Desarrollar una herramienta basada en inteligencia artificial que sintetice ...
2. Página 5: los momentos difíciles. También quiero reconocer a mi familia por su amor incondicional y apoyo cons...

🤔 Tu pregunta (o 